<a href="https://colab.research.google.com/github/karan-p2112/Car-Price-Predictor/blob/main/CarPricePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import pandas as pd
import numpy as np


# In[2]:


car=pd.read_csv("quikr_car.csv")


# In[3]:


car.head()


# In[4]:


car.shape


# In[5]:


car.info()


# In[6]:


car["company"].unique()


# # Improve quality of data
# - year has non-year values
# - year object to int
# - price has Ask for price kinda thingy
# - price object to int
# - kms driven is not integer
# - kms driven has nan values
# - fuel type has nan values
# - car name pretty long, solve that issue

# # Cleaning of data

# In[7]:


backup= car.copy()


# In[8]:


car=car[car["year"].str.isnumeric()]


# In[9]:


car["year"]= car["year"].astype(int)


# In[10]:


car=car[car["Price"]!= "Ask For Price"]


# In[11]:


car["Price"] = car["Price"].str.replace(',','').astype(int)


# In[12]:


car.info()


# In[13]:


car['kms_driven']= car["kms_driven"].str.split(' ').str.get(0).str.replace(',','')


# In[14]:


car= car[car['kms_driven'].str.isnumeric()]


# In[15]:


car.loc[:, "kms_driven"] = car["kms_driven"].astype(int)


# In[16]:


car.info()


# In[17]:


car= car[~car['fuel_type'].isna()]


# In[18]:


car['name']= car['name'].str.split(" ").str.slice(0,3).str.join(' ')


# In[19]:


car["name"]


# In[20]:


car.reset_index(drop=True)


# In[21]:


car.describe()


# In[22]:


car=car[car["Price"]<6e6].reset_index(drop=True)


# In[23]:


car


# In[24]:


car.to_csv("Cleaned Car.csv")


# # MODEL

# In[25]:


X = car.drop(columns="Price")
y=car["Price"]


# In[26]:


X


# In[27]:


y


# In[ ]:





# In[28]:


from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


# In[29]:


from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


# In[30]:


ohe= OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])


# In[31]:


column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']), remainder = 'passthrough')


# In[32]:


lr = LinearRegression()


# In[33]:


pipe= make_pipeline(column_trans,lr)


# In[34]:


pipe.fit(X_train,y_train)


# In[35]:


y_pred= pipe.predict(X_test)


# In[36]:


y_pred


# In[37]:


r2_score(y_test,y_pred)


# In[38]:


scores=[]
for i in range (1000):
    X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=i)
    lr = LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred= pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))


# In[39]:


np.argmax(scores)


# In[40]:


scores[np.argmax(scores)]


# In[41]:


X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
lr = LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred= pipe.predict(X_test)
r2_score(y_test,y_pred)


# In[42]:


import pickle


# In[43]:


pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))


# In[44]:


pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']], columns=['name','company','year','kms_driven','fuel_type']))


# In[ ]:






FileNotFoundError: ignored